In [9]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import utils
import json
from joblib import Parallel, delayed
from time import sleep

from importlib import reload
reload(utils)

<module 'utils' from '/usr/local/google/home/raulramos/projects/text-embeddings-benchmark/utils.py'>

In [2]:
# clone repo https://github.com/amazon-science/esci-data
# previously install git-lfs to download the data files when cloning

ESCI_DATASET_ROOT = '/usr/local/google/home/raulramos/projects/esci-data'

In [3]:
with open('../../secrets/apikeys.json') as f:
    GEMINI_API_KEY = json.load(f)['GEMINI_API_KEY']

In [4]:
# set locale

locale = 'us'

In [5]:
# load ground truth
dgt = utils.load_examples(ESCI_DATASET_ROOT=ESCI_DATASET_ROOT, locale=locale)

In [6]:
dgt.head()

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color,relevance
0,0,revent 80 cfm,0,B000MOO21W,us,I,0,1,train,Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceil...,None,WhisperCeiling fans feature a totally enclosed...,Panasonic,White,1
1,1,revent 80 cfm,0,B07X3Y6B1V,us,E,0,1,train,Homewerks 7141-80 Bathroom Fan Integrated LED ...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,80 CFM,4
2,2,revent 80 cfm,0,B07WDM7MQQ,us,E,0,1,train,Homewerks 7140-80 Bathroom Fan Ceiling Mount E...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,White,4
3,3,revent 80 cfm,0,B07RH6Z8KW,us,E,0,1,train,Delta Electronics RAD80L BreezRadiance 80 CFM ...,This pre-owned or refurbished product has been...,Quiet operation at 1.5 sones\nBuilt-in thermos...,DELTA ELECTRONICS (AMERICAS) LTD.,White,4
4,4,revent 80 cfm,0,B07QJ7WYFQ,us,E,0,1,train,Panasonic FV-08VRE2 Ventilation Fan with Reces...,None,The design solution for Fan/light combinations...,Panasonic,White,4


In [10]:
queries = dgt.groupby('query_id').first()[['query']]

In [11]:
products = dgt.groupby('product_id').first()[['product_title', 'product_description', 'product_bullet_point', 'product_brand', 'product_color']]
products = pd.DataFrame([{product_id: utils.get_product_string(pi) for product_id,pi in tqdm(products.iterrows())}], index=['product']).T

1215851it [01:34, 12850.08it/s]


In [12]:
queries.shape, products.shape

((97345, 1), (1215851, 1))

In [ ]:
x = utils.xget_gemini_batch_embeddings(queries['query'].values[:1000], GEMINI_API_KEY)

In [46]:
from importlib import reload
reload(utils)

<module 'utils' from '/usr/local/google/home/raulramos/projects/text-embeddings-benchmark/utils.py'>

In [58]:
queries['embeddings'] = utils.get_gemini_batch_embeddings_with_joblib(queries['query'].values, GEMINI_API_KEY, n_jobs=1)

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:   14.5s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:   56.9s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:  2.1min
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:  3.7min
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:  5.7min
[Parallel(n_jobs=1)]: Done 647 tasks      | elapsed:  8.2min
[Parallel(n_jobs=1)]: Done 881 tasks      | elapsed: 11.1min
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 974/974 [00:00<00:00, 17147.85it/s]


In [59]:
locale

'us'

In [60]:
queries.to_parquet(f'gemini/queries-{locale}.parquet')

In [65]:
products['embeddings'] = utils.get_gemini_batch_embeddings_with_joblib(products['product'].values, GEMINI_API_KEY, n_jobs=1)

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:   14.3s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:   58.3s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:  2.2min
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:  3.8min
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:  6.0min
[Parallel(n_jobs=1)]: Done 647 tasks      | elapsed:  8.8min
[Parallel(n_jobs=1)]: Done 881 tasks      | elapsed: 11.9min
[Parallel(n_jobs=1)]: Done 1151 tasks      | elapsed: 15.7min
[Parallel(n_jobs=1)]: Done 1457 tasks      | elapsed: 19.9min
[Parallel(n_jobs=1)]: Done 1799 tasks      | elapsed: 24.4min
[Parallel(n_jobs=1)]: Done 2177 tasks      | elapsed: 29.4min
[Parallel(n_jobs=1)]: Done 2591 tasks      | elapsed: 34.7min
[Parallel(n_jobs=1)]: Done 3041 tasks      | elapsed: 40.7min
[Parallel(n_jobs=1)]: Done 3527 tasks      | elapsed: 47.1min
[Parallel(n_jobs=1)]: Done 4049 tasks      | elapsed: 53.8min
[Parallel(n_jobs=1)]: Done 4607 tasks      | elapsed: 61.1min
[Parallel(n_job

In [69]:
products.to_parquet(f'gemini/products-{locale}.parquet')

In [ ]:
products.shape